### 朴素贝叶斯 鸢尾花


In [12]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

x,y = load_iris().data,load_iris().target
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1,test_size=50)
len(x_test)


#训练模型
model = GaussianNB()
model.fit(x_train,y_train)

#评估
pred = model.predict(x_test)
print('预测集数据的预测标签是：',pred)
print('测试机数据的真实标签是:',y_test)
acc = accuracy_score(y_test,pred)
print('预测的准确率是：',acc)
sum(pred!=y_test)

预测集数据的预测标签是： [0 1 1 0 2 2 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 2 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 1 2 1 2 0 0 0 1]
测试机数据的真实标签是: [0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 1 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 2 2 1 2 0 0 0 1]
预测的准确率是： 0.94


3

# 垃圾邮件

In [3]:
import os 
normalFileList = os.listdir('../bayes/normal')
spamFileList = os.listdir('../bayes/spam')
print('正常邮件列表为：',normalFileList)
print('垃圾邮件列表为：',spamFileList)

正常邮件列表为： ['normal-mail1.txt', 'normal-mail2.txt', 'normal-mail3.txt', 'normal-mail4.txt', 'normal-mail5.txt', 'normal-mail6.txt', 'normal-mail7.txt', 'normal-mail8.txt', 'normal-mail9.txt']
垃圾邮件列表为： ['spam-mail1.txt', 'spam-mail2.txt', 'spam-mail3.txt', 'spam-mail4.txt', 'spam-mail5.txt', 'spam-mail6.txt', 'spam-mail7.txt', 'spam-mail8.txt', 'spam-mail9.txt']


In [6]:
stopList = []
for line in open('../bayes/stopwords.txt','r',encoding='utf-8'):
    stopList.append(line[:len(line) - 1])
print(stopList)

['啊', '阿', '哎', '哎呀', '唉', '于是', '还']


In [ ]:
import jieba